In [1]:
# Standard kaggle packages
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
! pip install -U accelerate
! pip install xformers
! pip install vaderSentiment
! pip install transformers
! pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 44.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 68.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
from spacy import displacy
import spacy
import nltk
import pyarrow.compute as pc
import pyarrow as pa
import transformers 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
nlp = spacy.load("en_core_web_sm")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
dataset = load_dataset("webis/tldr-17", split='train')

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to /root/.cache/huggingface/datasets/webis___reddit/default/1.0.0/bd1bf9097540c9101f329c123d12c6c6a042f65e5f0ab7f9bbabb0a54d3c840e. Subsequent calls will reuse this data.


In [5]:
def has_entity_type(text):
    doc = nlp(text)
    for token in doc:
        if token.ent_type_ == "GPE":
            return True
    return False

def add_bool(example):
    example['GPE'] = has_entity_type(example['body'])
    return example

In [6]:
subset = dataset.select(range(200000))

In [7]:
with_gpe_bool = subset.map(add_bool, num_proc=5)

#0:   0%|          | 0/20 [00:00<?, ?ex/s]

#2:   0%|          | 0/20 [00:00<?, ?ex/s]

#1:   0%|          | 0/20 [00:00<?, ?ex/s]

#3:   0%|          | 0/20 [00:00<?, ?ex/s]

#4:   0%|          | 0/20 [00:00<?, ?ex/s]

In [8]:
with_gpe = with_gpe_bool.filter(lambda example: example['GPE']==True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

In [14]:
def tester(input):
    input_full = f'I want to label texts. The available labels are: "Culinaric", "Landscape", "Culture", "Weather", "Safety", "None". Label the following text: {input}' 
    inputs = tokenizer(input_full, return_tensors="pt")
    outputs = model.generate(**inputs)
    return(tokenizer.batch_decode(outputs, skip_special_tokens=True))

def add_label(df):
    df['label'] = tester(df['body'])
    return df

In [15]:
with_label = with_gpe.map(add_label)

  0%|          | 0/25 [00:00<?, ?ex/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


In [17]:
label_list = ["Culinaric", "Landscape", "Culture", "Weather", "Safety"]

In [18]:
only_label = with_label.filter(lambda x: x['label'][0] in label_list)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
!cp -r /kaggle/input/countries2 ./
!cp -r /kaggle/input/vectors ./
!cp -r /kaggle/input/usa-uk ./

In [21]:
countries = pd.read_excel('/kaggle/working/countries2/CountryCode.xlsx') 
country_list = countries['Country'].tolist()
usa_uk = pd.read_excel('/kaggle/working/usa-uk/USA_UK.xlsx') 
usa = usa_uk['USA'].tolist()
uk = usa_uk['United Kingdom'].tolist()

In [22]:
country_list = country_list[1:] # Antigua and Barbuda
delete = ['Antigua and Barbuda', 'Bosnia and Herzegovina', 'Burkina Faso', 
         "Côte d'Ivoire", 'Cabo Verde', 'Central African Republic', 'Costa Rica',
         'Dominican Republic', 'El Salvador', 'Equatorial Guinea', 'Eswatini',
         'Guinea-Bissau', 'Holy See', 'Marshall Islands', 'New Zealand', 'North Korea',
         'North Macedonia', 'Palestine State', 'Papua New Guinea', 'Saint Lucia',
         'San Marino', 'Saudi Arabia', 'Sierra Leone', 'Solomon Islands', 'South Africa',
         'South Korea', 'South Sudan', 'Sri Lanka', 'Trinidad and Tobago', 'United Arab Emirates',
         'United Kingdom', 'United States of America']
for c in delete:
    country_list.remove(c)

In [23]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('/kaggle/working/vectors/GoogleNews-vectors-negative300.bin', binary=True)

In [24]:
def has_entity_type(text, list_of_country):
    text_words = set(text.lower().split())
    for word in text_words:
        if word in list_of_country:
            return True
    return False

def country_bool(df):
    for country, words in country_dict.items():
        df[country] = has_entity_type(df['body'], words)
    return df

def country_dict(country_list):
    my_dict = {}
    for country in country_list:
        examples = ["country - country + city"]
        my_dict[country] = []
        
        for example in examples:
            words = example.split() 
            c_list = model.most_similar(positive=[country, words[4]], negative=words[2], topn=1000)
        c_list = [word.lower() for word, _ in c_list]
        c_list.append(country.lower())
        my_dict[country] = c_list
    
    return my_dict

In [ ]:
usa = [word.lower() for word in usa]
uk = [word.lower() for word in uk]

In [25]:
country_dict = country_dict(country_list)
country_dict['United Kingdom'] = uk
country_dict['USA'] = usa

In [26]:
country_df = only_label.map(country_bool, num_proc=5)

#0:   0%|          | 0/2 [00:00<?, ?ex/s]

#1:   0%|          | 0/2 [00:00<?, ?ex/s]

#2:   0%|          | 0/1 [00:00<?, ?ex/s]

#3:   0%|          | 0/1 [00:00<?, ?ex/s]

#4:   0%|          | 0/1 [00:00<?, ?ex/s]

In [27]:
from datasets import Dataset

def drop_rows_with_no_true(dataset, columns):
    new_rows = []
    for row in dataset:
        if any(row[col] for col in columns):
            new_rows.append(row)
    dataframe = pa.Table.from_pandas(pd.DataFrame(new_rows))
    return Dataset.from_pandas(dataframe.to_pandas())

In [28]:
country_df_short = drop_rows_with_no_true(country_df, country_list)

In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
def vader_sentiment(df):
    df['vader'] = analyzer.polarity_scores(df['body'])['compound']
    return df

In [30]:
df_sent = country_df_short.map(vader_sentiment)

  0%|          | 0/2 [00:00<?, ?ex/s]

In [31]:
df_sent.save_to_disk('/kaggle/working/final_df')